<a href="https://colab.research.google.com/github/birdx0810/LearningTensorFlow/blob/main/TensorFlow___From_1_To_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From TensorFlow 1.x to 2.x and PyTorch
This is notebook is basically an implementation of a basic multi-layer perceptron using TF1.0, TF2.0 and PyTorch method for MNIST dataset classification. The results should be similar.

In [1]:
import time

# Import MINST data 
from tensorflow.keras.datasets import mnist
from tqdm import trange

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:
# Set Model Parameters
learning_rate = 1e-5 
epochs = 10
batch_size = 32
num_labels = 10

In [3]:
# Set random seed
np.random.seed(42)

In [4]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x = train_x.reshape(train_x.shape[0], train_x.shape[1]*train_x.shape[2])
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1]*test_x.shape[2])

label_binarizer = LabelBinarizer()
label_binarizer.fit(range(num_labels))

train_onehot = label_binarizer.transform(train_y)
test_onehot = label_binarizer.transform(test_y)

In [5]:
print(train_x.shape)

(60000, 784)


In [6]:
def get_batch(x, y, iteration, batch_size):
    start = iteration * batch_size
    end = start + batch_size

    x_mb = x[start:end]
    y_mb = y[start:end]
  
    return x_mb, y_mb

## TensorFlow 1.0


In [7]:
import tensorflow.compat.v1 as tf
tf.set_random_seed(42)
tf.disable_v2_behavior()

# tf Graph Input 
x = tf.placeholder("float", [None, 784])  # mnist data image of shape 28*28 = 784 
y = tf.placeholder("float", [None, 10])   # 0-9 digits recognition => 10 classes

# Set model weight and bias 
W = tf.Variable(tf.ones([784, 10]))      # 784 -> 10 
b = tf.Variable(tf.zeros([10]))

logits = tf.matmul(x, W) + b
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))

pred = tf.nn.softmax(logits)
acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(y,1), predictions=tf.argmax(pred,1))

# Optimize model using gradient descent
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
# Initializing the variables 
init_global = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()

# Launch the graph 
with tf.Session() as sess:
    sess.run(init_global)
    sess.run(init_local)
  
    # Training cycle
    start = time.time()
    for epoch in range(epochs):
        total_loss = 0.
        iterations = int(len(train_x)/batch_size)
        # Loop over all batches
        for i in trange(iterations):
            # Get mini batch
            batch_xs, batch_ys = get_batch(train_x, train_onehot, i, batch_size)
      
            # Fit training using batch data 
            _, batch_loss = sess.run([optimizer, loss], feed_dict={x: batch_xs, y: batch_ys})

            total_loss += batch_loss/iterations

        # Display logs per epoch step
        total_loss = total_loss
        print(f"\nEpoch:{epoch+1}\ttotal loss={total_loss}")
    
    _, accuracy = sess.run([acc, acc_op], feed_dict={x: test_x, y: test_onehot})
    print(f"model acc: {accuracy}")
    print(f"Time taken: {time.time() - start} sec")

  7%|▋         | 135/1875 [00:00<00:01, 1343.63it/s]


Epoch:1	total loss=0.38425698899527416


  7%|▋         | 139/1875 [00:00<00:01, 1385.90it/s]


Epoch:2	total loss=0.2992388392756381


  7%|▋         | 139/1875 [00:00<00:01, 1378.66it/s]

...


Epoch:8	total loss=0.26952694824635987


  8%|▊         | 141/1875 [00:00<00:01, 1408.10it/s]


Epoch:9	total loss=0.26775324675391127


100%|██████████| 1875/1875 [00:01<00:00, 1424.03it/s]


Epoch:10	total loss=0.26621821887244795
model acc: 0.9172999858856201
Time taken: 13.767048120498657 sec


## TensorFlow 2.0

In [9]:
# Import relevant packages
import tensorflow as tf
tf.random.set_seed(42)

# First define the model 
model = tf.keras.Sequential()

# Add the dense layer to the model
model.add(tf.keras.layers.Dense(
    units=num_labels, 
    activation="softmax", 
    input_shape=(784,), 
    weights=[np.ones([784, 10]), np.zeros(10)]
))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Define loss and optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
criterion = tf.keras.losses.CategoricalCrossentropy()

model.compile(optimizer=optimizer, metrics=["accuracy"], loss=criterion)

In [11]:
start = time.time()
train_history = model.fit(
    train_x, train_onehot, 
    batch_size=batch_size, epochs=epochs
)
print(f"Time taken: {time.time() - start} sec")

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3768 - acc: 0.8963
Epoch 2/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2908 - acc: 0.9184
Epoch 3/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.2794 - acc: 0.9230
Epoch 4/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.2739 - acc: 0.9241
Epoch 5/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.2694 - acc: 0.9262
Epoch 6/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2656 - acc: 0.9268
Epoch 7/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.2634 - acc: 0.9273
Epoch 8/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.2616 - acc: 0.9275
Epoch 9/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.2589 - acc: 0.9286
Epoch 10/10
60000/60000 [==============================] - 

In [12]:
loss, accuracy = model.evaluate(test_x, test_onehot)
print(f"Acc: {accuracy}")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Acc: 0.9207000136375427


## PyTorch

In [31]:
import torch

torch.manual_seed(42)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(784, 10)
        torch.nn.init.ones_(self.linear.weight)
        torch.nn.init.zeros_(self.linear.bias)
        self.softmax = torch.nn.Softmax()
        
    def forward(self, X):
        logits = self.linear(X)
        pred = self.softmax(logits)
        
        return pred

In [32]:
model = Model()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [33]:
start = time.time()
# Training Loop
for epoch in range(epochs):
    total_loss = 0.
    iterations = int(len(train_x)/batch_size)
    # Loop over all batches
    for i in trange(iterations):
        # Get mini batch
        batch_xs, batch_ys = get_batch(train_x, train_y, i, batch_size)
        
        batch_xs = torch.FloatTensor(batch_xs)
        batch_ys = torch.LongTensor(batch_ys)
        
        model.zero_grad()
        
        batch_ps = model(batch_xs)
        loss = criterion(batch_ps, batch_ys)
        loss.backward()
        optimizer.step()
        
        total_loss += loss
    print(f"\nEpoch:{epoch+1}\ttotal loss={total_loss/iterations}")
    
print(f"Time taken: {time.time() - start} sec")

  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-31-5db9b7eb952f>:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = self.softmax(logits)
 25%|██▍       | 465/1875 [00:00<00:00, 4643.40it/s]


Epoch:1	total loss=1.6154459714889526


 25%|██▍       | 466/1875 [00:00<00:00, 4650.17it/s]


Epoch:2	total loss=1.5644217729568481


 25%|██▍       | 460/1875 [00:00<00:00, 4592.61it/s]

...


Epoch:8	total loss=1.5421299934387207


 24%|██▍       | 446/1875 [00:00<00:00, 4453.00it/s]


Epoch:9	total loss=1.5407434701919556


100%|██████████| 1875/1875 [00:00<00:00, 4421.22it/s]


Epoch:10	total loss=1.539556860923767
Time taken: 4.27206826210022 sec


In [34]:
model.eval()
test_x = torch.FloatTensor(test_x)
test_p = model(test_x)
test_p = test_p.detach().numpy().argmax(-1)

<ipython-input-31-5db9b7eb952f>:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = self.softmax(logits)


In [35]:
accuracy = accuracy_score(test_y, test_p)
print(f"Acc: {accuracy}")

Acc: 0.9256
